#Hi Welcome to The NoteBook 
###We will implement tokenization API in keras to encode sentences then we will use embedding to make sense of tokens and classify using embeddings 
##Data used : yelp_review from tensorflow datasets(tfds)


#Import


In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import io
print(tf.__version__)

2.2.0


#Load The dataset from tfds

In [2]:
yelp , info = tfds.load("yelp_polarity_reviews", with_info=True, as_supervised=True)

train_data,test_data = yelp["train"],yelp["test"]
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode("utf8"))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incompletePWWEZZ/yelp_polarity_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0.incompletePWWEZZ/yelp_polarity_reviews-test.tfrecord


Dataset yelp_polarity_reviews downloaded and prepared to /root/tensorflow_datasets/yelp_polarity_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


#Hyperparameter for training

In [3]:
vocab_size = 100000
embedding_dim = 64
max_length = 150
trunc_type = "post" #want to preserve the initial words
oov_token = "UNK"

In [4]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding="post")

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

#Function to retrive words from index and get the sentence

In [5]:
def decode_review(text):
  """ it will give you the original text"""
  reverse_dictorary = dict([(value,key) for (key,value) in word_index.items()])
  return " ".join([reverse_dictorary.get(i,"UNK") for i in text])

#test
print(decode_review(padded[3]))
print(training_sentences[3])
#assert(decode_review(padded[3])==training_sentences[3])

i wish i would have read megan p's review before i decided to cancel my dinner reservations because i was offered a comp dinner at beso n ntwo of my girlfriends and i were walking around fashion show mall as a promoter approaches us to ask if we'd like to go to eve nightclub that night we wanted to go to haze and xs so we declined the promoter by the way his name is andre offers us a comp dinner at beso eva longoria's restaurant at crystals mall i guess what's the catch right is there a comp limit and we have to pay after we hit the limit no it's a set dinner he sets everything up and promises he'll even walk us into haze and xs afterwards which i actually believed n nhere's a review for andre nhe tells you to call him when you're at the
I wish I would have read Megan P's review before I decided to cancel my dinner reservations because I was offered a COMP dinner at Beso.  \n\nTwo of my girlfriends and I were walking around Fashion Show Mall as a promoter approaches us to ask if we'd l

#Model Building

In [6]:
#difine the model
def model(vocab_size,embedding_dim,max_length):
  model = tf.keras.Sequential([
                              tf.keras.layers.Embedding(vocab_size,embedding_dim, input_length=max_length),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(12,activation="relu"),
                              tf.keras.layers.Dense(24,activation="relu"),
                              tf.keras.layers.Dense(1,activation="sigmoid")

])
  model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
  model.summary()
  return model

In [7]:
model = model(vocab_size,embedding_dim,max_length)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 64)           6400000   
_________________________________________________________________
flatten (Flatten)            (None, 9600)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                115212    
_________________________________________________________________
dense_1 (Dense)              (None, 24)                312       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 6,515,549
Trainable params: 6,515,549
Non-trainable params: 0
_________________________________________________________________


#Training Parameters

In [9]:
num_epochs = 1
history=model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

17500/17500 [==============================] - 1237s 71ms/step - loss: 0.1546 - accuracy: 0.9395 - val_loss: 0.3596 - val_accuracy: 0.8245


#Visualization of training

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

#scrap the weights

In [11]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(100000, 64)


understand it we give 100000 vocab size and 64 dimention for each of the weights

#Download weights for visualization

In [12]:
reverse_dictorary = dict([(value,key) for (key,value) in word_index.items()])
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_dictorary[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [13]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
sentence = "I really like this product though it is not that good."
sequence = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequences, maxlen=max_length, padding="post", truncating=trunc_type)
print(model.predict(padded))
print(sequence)
print(padded)

[[0.97201717]
 [0.00792843]
 [0.24799585]
 ...
 [0.9991586 ]
 [0.5070185 ]
 [0.9881825 ]]
[[4, 62, 39, 17, 1311, 196, 9, 12, 23, 14, 35]]
[[    2 16671  1614 ...   851    14    13]
 [  732   620    40 ...     0     0     0]
 [   23    26    38 ...     0     0     0]
 ...
 [  334    53 10225 ...     0     0     0]
 [   16   103     5 ...     0     0     0]
 [   15   370    96 ...     5   125    78]]


#Visualization:
follow [this link](https://projector.tensorflow.org/) and upload vec.tsv and meta.tsv to visualization use t-sne as decomposition method

#Written by : SATYAJIT MAITRA
## follow github: https://github.com/MachineLearningWithHuman/
## Email for work : ssatyajitmaitra@gmail.com
##learn with me on youtube: https://www.youtube.com/channel/UCiWd572-4LeH0IqJ5A7LavA/
### please give a star if it helps you 